# Correct minian metadata if needed

In [ ]:
cd "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/minian"

In [ ]:
minian_path = "."
import os

import sys
import warnings
import itertools as itt
import numpy as np
import xarray as xr
import holoviews as hv
import pandas as pd
from pathlib import Path
from holoviews.operation.datashader import datashade, regrid
from dask.diagnostics import ProgressBar
sys.path.append(minian_path)
from IPython.display import display
from ipyfilechooser import FileChooser
from holoviews.operation.datashader import datashade, regrid


from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)
from minian.visualization import AlignViewer
import matplotlib.pyplot as plt
from pathlib import Path


import warnings
warnings.filterwarnings("ignore")

In [ ]:
try:
    %store -r dpath
except:
    print("data not in strore")
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"


# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Choose Mice Folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

Load metadata

In [ ]:
minian_ds_path_Origin = os.path.join(dpath, "minian")

def correct_meta(ds,  minian_ds_path_Origin):
    path_obj =Path(minian_ds_path_Origin)
    #print(len(path_obj.parts))
    if len(path_obj.parts)==12: # Not a DrugExperiment with Subsessions       
        ds['session'] = path_obj.parents[0].name
        ds['animal'] = path_obj.parents[4].name
    elif len(path_obj.parts)==11: # Not a DrugExperiment with No Subsessions
        ds['session'] = path_obj.parents[1].name.split("_")[-1]
        ds['animal'] = path_obj.parents[3].name
    print(f"['animal: {ds['animal'].values}']")
    print(f"['session: {ds['session'].values}']")
    return ds

minian_ds = open_minian(minian_ds_path_Origin, post_process=correct_meta)

max_proj = minian_ds['max_proj']
max_projnp = minian_ds['max_proj'].values

"""
max_projnp = np.rot90(max_projnp) # rotate 90°degrees clockwise
max_projnp = np.rot90(max_projnp) # rotate 90°degrees clockwise
max_projnp = np.rot90(max_projnp) # rotate 90°degrees clockwise

max_projnp = np.flip(max_projnp, (0,1)) #flip horizontally & vertically
"""
max_proj.values = max_projnp


Display intial template

In [ ]:
output_size = 100
hv.notebook_extension('bokeh', width=100)
hv.output(size=int(output_size * 0.6))
opts_im = {'frame_height': 500,
    'frame_width': 500, 'cmap': 'flag'}
max_proj = minian_ds['max_proj'].rename('temps')
hv_max_proj = hv.Dataset(max_proj).to(hv.Image, ['width', 'height'])
hv_max_proj.opts(**opts_im).relabel("Initial Templates")

Rotate as much as needed or flip horizontally &/or vertically

In [ ]:
max_projnp = np.rot90(max_projnp) # rotate 90°degrees clockwise
#max_projnp = np.flip(max_projnp, (0,1)) #flip horizontally & vertically

max_proj.values = max_projnp

Display modified template

In [ ]:
hv.output(size=int(output_size * 0.6))
opts_im = {'frame_height': 500,
    'frame_width': 500, 'cmap': 'flag'}
hv_max_proj = hv.Dataset(max_proj).to(hv.Image, ['width', 'height'])
hv_max_proj.opts(**opts_im).relabel("Modified templates")

Save metadata

In [ ]:
param_save_minian = {
    "dpath": minian_ds_path_Origin,
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True
}

max_proj = save_minian(max_proj.rename("max_proj"), **param_save_minian)
